In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys

%pylab inline
pylab.rcParams['figure.figsize'] = (20.0, 10.0)

%load_ext autoreload
%autoreload 2

sys.path.append('..')

import isolation
import sample_players
import run_match
import my_baseline_player as custom

Populating the interactive namespace from numpy and matplotlib


In [2]:
isolation.isolation._ACTIONSET

{<Action.SSE: -27>,
 <Action.SSW: -25>,
 <Action.ESE: -15>,
 <Action.WSW: -11>,
 <Action.ENE: 11>,
 <Action.WNW: 15>,
 <Action.NNE: 25>,
 <Action.NNW: 27>}

## A table could be made to have "state-action" pairs as key, and winning frequency as value

In [3]:
book = dict()

In [4]:
class ProbePlayer(custom.CustomPlayer):
    """ A class that is used to get statistics about the game."""
    def __init__(self, player_id):
        super().__init__(player_id)
        self.book_depth = 4
        self.context = dict()
        self.context['book'] = dict()
    
    def get_action(self, state):
        print('Player {} has played like this: {}'.format(self.player_id,
                                                           self.context['book']))
        depth = 1
        while True:
            action = self.alpha_beta_search(state, depth)
            if depth <= self.book_depth:
                self.context['book'][state.ply_count] = (state.board, state.locs, action)
            self.queue.put(action)
            depth += 1

In [5]:
state = isolation.isolation.Isolation()

In [6]:
state

Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None))

In [7]:
state.board

41523161203939122082683632224299007

In [8]:
# Custom vs Custom
from time import time

run_match.TIME_LIMIT = 150

num_rounds = 1
num_procs = 4
agent1 = isolation.Agent(ProbePlayer, "ProbePlayer1")
agent2 = isolation.Agent(ProbePlayer, "ProbePlayer2")

tic = time()
wins, num_games = run_match.play_matches(agent1, 
                                         agent2, 
                                         num_rounds, 
                                         num_procs, 
                                         fair_matches=False)
toc = time()
print('Wins: {}, Games: {}, Ratio: {}'.format(wins, 
                                              num_games, 
                                              wins / num_games))
print('Total time: {}, Time per game: {}'.format((toc-tic), (toc-tic)/num_games))

Running 2 games:
Player 0 has played like this: {}
Player 0 has played like this: {}
-+
Wins: 1, Games: 2, Ratio: 0.5
Total time: 0.19027996063232422, Time per game: 0.09513998031616211


In [9]:
root = isolation.isolation.Isolation()

In [10]:
len(root.actions())

99

In [11]:
def get_full_tree(state, depth, s_a_set=None):
    if s_a_set == None:
        s_a_set = set()
    for action in state.actions():
        s_a_set.add((state, action))
        if depth > 1:
            s_a_set |= get_full_tree(state.result(action), depth - 1, s_a_set)
    return s_a_set

In [12]:
root = isolation.isolation.Isolation()
tree = get_full_tree(root, 1)

In [13]:
len(tree)

99

In [14]:
tree

{(Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  0),
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  1),
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  2),
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  3),
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  4),
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  5),
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  6),
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  7),
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  8),
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  9),
 (Isolation(board=41523161203939122082683632224299007, ply_c

In [15]:
def get_empty_book(state, depth):
    tree = get_full_tree(state, depth)
    return {key: 0 for key in tree}

In [16]:
root = isolation.isolation.Isolation()
book = get_empty_book(root, 1)
book

{(Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  0): 0,
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  1): 0,
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  2): 0,
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  3): 0,
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  4): 0,
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  5): 0,
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  6): 0,
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  7): 0,
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  8): 0,
 (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)),
  9): 0,
 (Isolation(board=415231612039

In [17]:
from time import time

times = list()
for depth in range(1,5):
    tic = time()
    root = isolation.isolation.Isolation()
    tree = get_full_tree(root, depth)
    toc = time()
    times.append(toc-tic)

In [18]:
times

[0.00010800361633300781,
 0.012609004974365234,
 0.2736680507659912,
 1.9176537990570068]

In [19]:
tree

{(Isolation(board=41198642638191437159754184456660991, ply_count=3, locs=(108, 83)),
  <Action.ESE: -15>),
 (Isolation(board=40143957350286345584047904549496831, ply_count=3, locs=(106, 110)),
  <Action.WSW: -11>),
 (Isolation(board=41198632737088974515941243613079551, ply_count=3, locs=(108, 83)),
  <Action.ESE: -15>),
 (Isolation(board=41198484184284260270308255718172671, ply_count=3, locs=(108, 83)),
  <Action.ESE: -15>),
 (Isolation(board=41523161203939122082683630076618751, ply_count=3, locs=(31, 17)),
  <Action.ESE: -15>),
 (Isolation(board=41523161203937941491027730440906719, ply_count=3, locs=(45, 5)),
  <Action.WNW: 15>),
 (Isolation(board=41523161203939122082683626855589879, ply_count=3, locs=(30, 32)),
  <Action.WSW: -11>),
 (Isolation(board=41523161203939122073672032775563263, ply_count=3, locs=(31, 53)),
  <Action.SSW: -25>),
 (Isolation(board=41520625901529739768237212624218111, ply_count=3, locs=(55, 101)),
  <Action.ENE: 11>),
 (Isolation(board=4152316120393912207818003

In [20]:
agent_names = ('CustomPlayer1', 'CustomPlayer2')
agent1 = isolation.Agent(custom.CustomPlayer, agent_names[0])
agent2 = isolation.Agent(custom.CustomPlayer, agent_names[1])
agents = (agent1, agent2)

state = isolation.isolation.Isolation()
time_limit = 150
match_id = 0
winner, game_history, match_id = isolation.play((agents,
                                                state,
                                                time_limit,
                                                match_id))

In [21]:
winner.name

'CustomPlayer1'

In [22]:
agent_names.index(winner.name)

0

In [23]:
game_history

[15,
 1,
 <Action.NNE: 25>,
 <Action.NNE: 25>,
 <Action.NNE: 25>,
 <Action.WNW: 15>,
 <Action.WSW: -11>,
 <Action.SSW: -25>,
 <Action.NNE: 25>,
 <Action.WSW: -11>,
 <Action.WSW: -11>,
 <Action.NNE: 25>,
 <Action.SSW: -25>,
 <Action.NNE: 25>,
 <Action.WSW: -11>,
 <Action.WSW: -11>,
 <Action.NNE: 25>,
 <Action.SSW: -25>,
 <Action.NNE: 25>,
 <Action.ESE: -15>,
 <Action.NNE: 25>,
 <Action.NNW: 27>,
 <Action.WSW: -11>,
 <Action.WNW: 15>,
 <Action.ESE: -15>,
 <Action.NNE: 25>,
 <Action.SSW: -25>,
 <Action.WSW: -11>,
 <Action.NNW: 27>,
 <Action.NNE: 25>,
 <Action.NNE: 25>,
 <Action.SSE: -27>,
 <Action.ESE: -15>,
 <Action.ENE: 11>,
 <Action.ESE: -15>,
 <Action.NNE: 25>,
 <Action.SSW: -25>,
 <Action.WNW: 15>,
 <Action.SSW: -25>,
 <Action.SSW: -25>,
 <Action.SSW: -25>,
 <Action.SSW: -25>,
 <Action.WNW: 15>,
 <Action.WNW: 15>,
 <Action.WSW: -11>,
 <Action.SSE: -27>,
 <Action.WNW: 15>,
 <Action.WSW: -11>,
 <Action.ENE: 11>,
 <Action.SSE: -27>,
 <Action.SSW: -25>,
 <Action.NNE: 25>,
 <Action.WNW: 1

In [24]:
len(game_history)

78

In [25]:
b = True
2*b-1

1

In [26]:
b = False
2*b-1

-1

In [33]:
def process_game_history(state, 
                         game_history, 
                         book, 
                         winner_id, 
                         active_player=0,
                         depth=4):
    """ Given an initial state, and a list of actions, this function iterates
    through the resulting states of the actions and updates count of wins in 
    the state/action book"""
    game_value = 2 * (active_player == winner_id) - 1
    curr_state = state  # It is a named tuple, so I think it is immutable. No need to copy.
    for num_action, action in enumerate(game_history):
        print('Checking {}'.format((curr_state, action)))
        if (curr_state, action) in book.keys():
            print('Updating {} with {}'.format((curr_state, action), game_value))
            book[(curr_state, action)] += game_value
        curr_state = curr_state.result(action)
        active_player = 1 - active_player
        game_value = 2 * (active_player == winner_id) - 1
        # Break on depth equal to book
        if num_action >= depth - 1:
            break

In [36]:
depth = 4
root = isolation.isolation.Isolation()
book = get_empty_book(root, depth)
print(sum(abs(value) for value in book.values()))
process_game_history(root, 
                     game_history, 
                     book, 
                     agent_names.index(winner.name),
                     active_player=0,
                     depth=depth)

0
Checking (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)), 15)
Updating (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)), 15) with 1
Checking (Isolation(board=41523161203939122082683632224266239, ply_count=1, locs=(15, None)), 1)
Updating (Isolation(board=41523161203939122082683632224266239, ply_count=1, locs=(15, None)), 1) with -1
Checking (Isolation(board=41523161203939122082683632224266237, ply_count=2, locs=(15, 1)), <Action.NNE: 25>)
Updating (Isolation(board=41523161203939122082683632224266237, ply_count=2, locs=(15, 1)), <Action.NNE: 25>) with 1
Checking (Isolation(board=41523161203939122082682532712638461, ply_count=3, locs=(40, 1)), <Action.NNE: 25>)
Updating (Isolation(board=41523161203939122082682532712638461, ply_count=3, locs=(40, 1)), <Action.NNE: 25>) with -1


In [38]:
sum(abs(value) for value in book.values())

4

### Let's fill the book using random actions for the first two.

In [ ]:
agent_names = ('CustomPlayer1', 'CustomPlayer2')
agent1 = isolation.Agent(custom.CustomPlayer, agent_names[0])
agent2 = isolation.Agent(custom.CustomPlayer, agent_names[1])
agents = (agent1, agent2)

state = isolation.isolation.Isolation()
time_limit = 150
match_id = 0
winner, game_history, match_id = isolation.play((agents,
                                                state,
                                                time_limit,
                                                match_id))

In [ ]:
from multiprocessing.pool import ThreadPool as Pool

# Initialization
depth = 4
time_limit = 150
book = get_empty_book(root, depth)
agent_names = ('CustomPlayer1', 'CustomPlayer2')
agent1 = isolation.Agent(custom.CustomPlayer, agent_names[0])
agent2 = isolation.Agent(custom.CustomPlayer, agent_names[1])
agents = (agent1, agent2)
root = isolation.isolation.Isolation()

In [ ]:
# Playing


In [39]:
# Updating the book

# Will update "depth" states per run.
process_game_history(root, 
                     game_history, 
                     book, 
                     agent_names.index(winner.name),
                     active_player=0,
                     depth=depth)

Checking (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)), 15)
Updating (Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(None, None)), 15) with 1
Checking (Isolation(board=41523161203939122082683632224266239, ply_count=1, locs=(15, None)), 1)
Updating (Isolation(board=41523161203939122082683632224266239, ply_count=1, locs=(15, None)), 1) with -1
Checking (Isolation(board=41523161203939122082683632224266237, ply_count=2, locs=(15, 1)), <Action.NNE: 25>)
Updating (Isolation(board=41523161203939122082683632224266237, ply_count=2, locs=(15, 1)), <Action.NNE: 25>) with 1
Checking (Isolation(board=41523161203939122082682532712638461, ply_count=3, locs=(40, 1)), <Action.NNE: 25>)
Updating (Isolation(board=41523161203939122082682532712638461, ply_count=3, locs=(40, 1)), <Action.NNE: 25>) with -1


In [48]:
import datetime as dt

timestamp = dt.datetime.now()
print(timestamp)
print('_'.join(timestamp.__str__().split()))

2018-06-01 11:22:05.525728
2018-06-01_11:22:05.525728


In [137]:
import datetime as dt
import pickle

# Saving the book
def save_book(book):
    timestamp = dt.datetime.now()
    filename = 'book' + timestamp.__str__().replace(' ', '_').replace(':','$') + '.pkl'
    with open(filename, 'wb') as file:
        pickle.dump(book, file, protocol=pickle.HIGHEST_PROTOCOL)

In [138]:
save_book(book)

In [139]:
import os
import re

In [140]:
pattern = 'book(.*)\.pkl'
book_list = [(re.match(pattern, filename).group(0), re.match(pattern, filename).group(1)) 
             for filename in os.listdir('.') 
             if re.match(pattern, filename) is not None]

In [141]:
book_list

[('book2018-06-01_11$47$10.006977.pkl', '2018-06-01_11$47$10.006977'),
 ('book2018-06-01_11$47$08.970941.pkl', '2018-06-01_11$47$08.970941'),
 ('book2018-06-01_11$49$40.780680.pkl', '2018-06-01_11$49$40.780680'),
 ('book2018-06-01_11$36$36.546475.pkl', '2018-06-01_11$36$36.546475'),
 ('book2018-06-01_11$47$07.726019.pkl', '2018-06-01_11$47$07.726019')]

In [142]:
date_str = book_list[0][1].replace('_', ' ').replace('$', ':')
date_str

'2018-06-01 11:47:10.006977'

In [143]:
dt.datetime.strptime(book_list[0][1], '%Y-%m-%d_%H$%M$%S.%f')

datetime.datetime(2018, 6, 1, 11, 47, 10, 6977)

In [144]:
# Does it pad with zeros in the right? (that is important)
timestamps = [dt.datetime.now() for _ in range(10000)]

In [129]:
timestamps

[datetime.datetime(2018, 6, 1, 11, 47, 13, 685595),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685603),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685604),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685605),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685606),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685607),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685608),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685609),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685610),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685611),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685612),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685613),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685614),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685615),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685616),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685617),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685618),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685619),
 datetime.datetime(2018, 6, 1, 11, 47, 13, 685620),
 datetime.da

In [130]:
# OK, it does

In [145]:
book_list

[('book2018-06-01_11$47$10.006977.pkl', '2018-06-01_11$47$10.006977'),
 ('book2018-06-01_11$47$08.970941.pkl', '2018-06-01_11$47$08.970941'),
 ('book2018-06-01_11$49$40.780680.pkl', '2018-06-01_11$49$40.780680'),
 ('book2018-06-01_11$36$36.546475.pkl', '2018-06-01_11$36$36.546475'),
 ('book2018-06-01_11$47$07.726019.pkl', '2018-06-01_11$47$07.726019')]

In [147]:
book_list = [
        (re.match(pattern, filename).group(0), 
         dt.datetime.strptime(re.match(pattern, filename).group(1), 
                              '%Y-%m-%d_%H$%M$%S.%f')) 
        for filename in os.listdir('.') 
        if re.match(pattern, filename) is not None
]

In [148]:
book_list

[('book2018-06-01_11$47$10.006977.pkl',
  datetime.datetime(2018, 6, 1, 11, 47, 10, 6977)),
 ('book2018-06-01_11$47$08.970941.pkl',
  datetime.datetime(2018, 6, 1, 11, 47, 8, 970941)),
 ('book2018-06-01_11$49$40.780680.pkl',
  datetime.datetime(2018, 6, 1, 11, 49, 40, 780680)),
 ('book2018-06-01_11$36$36.546475.pkl',
  datetime.datetime(2018, 6, 1, 11, 36, 36, 546475)),
 ('book2018-06-01_11$47$07.726019.pkl',
  datetime.datetime(2018, 6, 1, 11, 47, 7, 726019))]

In [149]:
max(book_list, key=lambda x: x[1])[0]

'book2018-06-01_11$49$40.780680.pkl'

In [277]:
import re
import os
from isolation.isolation import Isolation

def load_latest_book(depth=4):
    pattern = 'book(.*)\.pkl'
    book_list = [
        (re.match(pattern, filename).group(0), 
         dt.datetime.strptime(re.match(pattern, filename).group(1), 
                              '%Y-%m-%d_%H$%M$%S.%f')) 
        for filename in os.listdir('.') 
        if re.match(pattern, filename) is not None
    ]
    if len(book_list) == 0:
        return get_empty_book(Isolation(), depth)
    latest_name = max(book_list, key=lambda x: x[1])[0]
    with open(latest_name, 'rb') as file:
        latest_book = pickle.load(file)
    return latest_book

In [280]:
book = load_latest_book()

In [281]:
sum(abs(value) for value in book.values())

0

## Let's use some symmetries to reduce the search space. The symmetries will be valid for the 1st and 2nd move only (which are the most important, in the number of branches)

In [388]:
u = [1,2,3,4]
u.reverse()
print(u)

[4, 3, 2, 1]


In [424]:
def show_board(board, W, H):
    """
    This function shows a regular board (not a bitboard).
    Borders are not counted in W, so the board has W + 2.
    """
    for i in range(H-1, -1, -1):
        row = board[i*(W+2):(i+1)*(W+2)-2].copy()
        row.reverse()
        print(row)
#       print('  '.join([str(v) for v in row]))

In [434]:
W, H = 11, 9
board = [0]*(W+2)*(H)
show_board(board, W, H)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [436]:
W, H = 11, 9
board = [0]*(W+2)*(H)
board[14] = 1
show_board(board, W, H)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [437]:
W, H = 11, 9
board = [0]*(W+2)*(H)
board[0] = 1
show_board(board, W, H)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [433]:
W, H = 11, 9
board = [0]*(W+2)*(H)
board[(W+2)*H-3] = 1
show_board(board, W, H)

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


The order is not the same as in the game, but it is good enough to check for symmetries (they are... ugh... symmetric :P )

In [528]:
W, H = 11, 9
def h_symmetry(loc):
    if loc is None:
        return None
    row = (loc + 1) // (W + 2)
    col = loc % (W + 2)
    center = W // 2  + row
    return row * W + 2 * center - col

In [529]:
loc = 0
board = [0]*(W+2)*(H)
board[loc] = 1
print(h_symmetry(loc))
board[h_symmetry(loc)] = 2
show_board(board, W, H)

10
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [536]:
def v_symmetry(loc):
    if loc is None:
        return None
    row = (loc + 1) // (W + 2)
    col = loc % (W + 2)
    center = H // 2
    return (2 * center - row) * (W + 2) + col

In [539]:
loc = 14 + 13*2
board = [0]*(W+2)*(H)
board[loc] = 1
board[v_symmetry(loc)] = 2
show_board(board, W, H)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [546]:
def c_symmetry(loc):
    return h_symmetry(v_symmetry(loc))

In [548]:
loc = 14 + 13
board = [0]*(W+2)*(H)
board[loc] = 1
board[c_symmetry(loc)] = 2
show_board(board, W, H)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [549]:
loc = 4*13 + 6  # The fixed point...
board = [0]*(W+2)*(H)
board[loc] = 1
board[c_symmetry(loc)] = 2
show_board(board, W, H)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [550]:
q1 = list(range((4*13 + 6)//2 + 1))
board = [0]*(W+2)*(H)
for loc in q1:
    board[(W+2)*(loc//6) + loc%6] = 1
show_board(board, W, H)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]


In [551]:
board_s = [0]*(W+2)*(H)
for loc in range((W+2)*(H)-2):
    if board[loc] == 1:
        board_s[c_symmetry(loc)] = 1
show_board(board_s, W, H)

[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Let's get a dictionary of symmetric locations

In [552]:
# The first quadrant
mod = (W + 2) // 2 + 1
q1 = [(W+2)*(loc // mod) + loc % mod for loc in range(mod*(H//2 + 1))]

# Show it
board = [0]*(W+2)*(H)
for loc in q1:
    board[loc] = 1
show_board(board, W, H)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]


In [553]:
total = list(range((W+2)*(H)))
print('{} - {}'.format(len(board), len(total)))

117 - 117


In [554]:
sym = dict()
for loc in total:
    sym[loc] = [h_symmetry(loc), v_symmetry(loc), c_symmetry(loc)]

In [555]:
loc = 14
board = [0]*(W+2)*(H)
board[loc] = 1
for s in sym[loc]:
    board[s] = 2
show_board(board, W, H)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [556]:
game_history[0]

15

In [557]:
print(sym[14])

[22, 92, 100]


In [558]:
state = isolation.isolation.Isolation(locs=(92, None))
state.locs

(92, None)

In [559]:
tuple(map(v_symmetry, (92, 4)))

(14, 108)

In [560]:
from isolation.isolation import Isolation

def q1_symmetric(state, W=11, H=9):
    # The first quadrant
    mod = (W + 2) // 2 + 1
    q1 = [(W+2)*(loc // mod) + loc % mod for loc in range(mod*(H//2 + 1))]
    
    if state.locs[0] is None:
        return state
    if state.locs[0] in q1:
        return state
    # Horizontal
    if h_symmetry(state.locs[0]) in q1:
        return Isolation(board=state.board,
                         ply_count=state.ply_count,
                         locs=tuple(map(h_symmetry, state.locs)))
    # Vertical
    if v_symmetry(state.locs[0]) in q1:
        return Isolation(board=state.board,
                         ply_count=state.ply_count,
                         locs=tuple(map(v_symmetry, state.locs)))
    # Central
    if c_symmetry(state.locs[0]) in q1:
        return Isolation(board=state.board,
                         ply_count=state.ply_count,
                         locs=tuple(map(c_symmetry, state.locs)))

In [561]:
state = isolation.isolation.Isolation(locs=(92, None))
q1_symmetric(state)

Isolation(board=41523161203939122082683632224299007, ply_count=0, locs=(14, None))

In [562]:
state = isolation.isolation.Isolation(locs=(24, 102))
q1_symmetric(state)

In [563]:
state = isolation.isolation.Isolation(locs=(102, 78))
q1_symmetric(state)

OK, seems to work.

## Let's get the depth=4 q1 states

In [564]:
from isolation.isolation import Isolation

depth = 4
W, H = 11, 9
mod = (W + 2) // 2 + 1
q1 = [(W+2)*(loc // mod) + loc % mod for loc in range(mod*(H//2 + 1))]

tree = []
for loc in q1:
    tree += get_full_tree(Isolation(locs=(loc, None)), depth=depth - 1)
len(tree)

156396

In [565]:
full_tree = get_full_tree(Isolation(), depth=depth)
len(full_tree) / len(tree)

2.376576127266682

### Make simmetric actions

In [566]:
import isolation.isolation as iso

In [567]:
iso.S

-13

In [568]:
iso.Action.NNE

<Action.NNE: 25>

In [569]:
for action in iso.Action:
    print(str(action).split('.')[1])

NNE
ENE
ESE
SSE
SSW
WSW
WNW
NNW


In [570]:
action = iso.Action.NNE
code = str(action).split('.')[1]
code

'NNE'

In [571]:
cardinal_sym_h = {
    'N': 'N',
    'S': 'S',
    'E': 'W',
    'W': 'E'
}

cardinal_sym_v = {
    'N': 'S',
    'S': 'N',
    'E': 'E',
    'W': 'W'
}

cardinal_sym_c = {
    'N': 'S',
    'S': 'N',
    'E': 'W',
    'W': 'E'
}

actions_dict = {
    'N': iso.N,
    'S': iso.S,
    'E': iso.E,
    'W': iso.W,
}

In [572]:
list(map(lambda x: cardinal_sym_h[x], code))

['N', 'N', 'W']

In [573]:
action.value

25

In [574]:
def value2action(value):
    return next(a for a in iso.Action if a.value == value)

In [575]:
from functools import reduce

def action_symmetric(action, cardinal_sym):
    code = str(action).split('.')[1]
    action_value = reduce(lambda x,y: x + y, 
                          map(lambda x: actions_dict[cardinal_sym[x]], code))
    return value2action(action_value)

In [576]:
action_symmetric(action, cardinal_sym_h)

<Action.NNW: 27>

In [581]:
def sym_sa(s_a, loc_sym, cardinal_sym):
    """
    Symmetry for a (state, action) pair. 
    Don't use if this state is from move 3 or more.
    """
    state = s_a[0]
    action = s_a[1]
    
    new_board = iso._BLANK_BOARD
    new_locs = tuple(map(loc_sym, state.locs))
    if new_locs[0] is not None:
        new_board = new_board ^ (1 << (new_locs[0]))
    if new_locs[1] is not None:
        new_board = new_board ^ (1 << (new_locs[1]))
    new_state = Isolation(board=new_board,
                          ply_count=state.ply_count,
                          locs=new_locs)
    new_action = action_symmetric(action, cardinal_sym)
    return new_state, new_action

In [586]:
tree[1]

(Isolation(board=41523161203939122082666039904036863, ply_count=2, locs=(27, 44)),
 <Action.SSW: -25>)

In [583]:
sym_sa(tree[0], h_symmetry, cardinal_sym_h)

(Isolation(board=41523161203636890627779940570884095, ply_count=2, locs=(35, 78)),
 <Action.NNW: 27>)

In [590]:
print(iso.DebugState.from_state(tree[1][0]))


+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   | 2 |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   | 1 |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +



In [589]:
print(iso.DebugState.from_state(sym_sa(tree[1], c_symmetry, cardinal_sym_c)[0]))


+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   | 1 |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   | 2 |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +



In [578]:
board = tree[0][0]
db = iso.DebugState.from_state(board)

In [579]:
print(db)


+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
| 2 |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   | 1 |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +



In [326]:
tree[0][0].board == iso._BLANK_BOARD

False

In [580]:
state = tree[0][0]

new_board = iso._BLANK_BOARD
new_locs = tuple(map(h_symmetry, state.locs))
if new_locs[0] is not None:
    new_board = new_board ^ (1 << (new_locs[0]))
if new_locs[1] is not None:
    new_board = new_board ^ (1 << (new_locs[1]))
new_state = Isolation(board=new_board,
                      ply_count=state.ply_count,
                      locs=new_locs)
    
db = iso.DebugState.from_state(new_state)
print(new_locs)
print(state.locs)
print(db.bitboard_string)
print(db)

(35, 78)
(27, 88)
1111111111100111111111110011111111110001111111111100111111111110011111111111001011111111100111111111110011111111111

+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   | 2 |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   | 1 |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - + - + - +
|   |   |   |   |   |   |   |   |   |   |   |
+ - + - + - + - + - + - + - + - + - +

In [350]:
new_board = iso._BLANK_BOARD
new_board

41523161203939122082683632224299007

In [352]:
'{:b}'.format(new_board)

'1111111111100111111111110011111111111001111111111100111111111110011111111111001111111111100111111111110011111111111'

In [367]:

'{:b}'.format(1 << state.locs[0])

'1000000000000000000000000000'

### Conclusion: it is necessary to fix the order in the bitboard

## IDEA: Iterative Time Limit increase

## IDEA 2: Add the book info to the players and repeat the process

## IDEA 3: Play against Monte Carlo Player